In [10]:
import numpy as np
import torch
import chess

In [14]:
# convert a position to the tensor representation used by the nn
def board_2_tensor(board: chess.Board) -> torch.tensor:
    nums = np.empty(13, dtype=np.uint64)
    for i in range(6):
        nums[i] = int(board.pieces(i + 1, chess.WHITE))
        nums[i + 6] = int(board.pieces(i + 1, chess.BLACK))

    nums[12] = 1 << board.ep_square if board.has_legal_en_passant() else 0

    bits = np.unpackbits(nums.view(np.uint8))
    return torch.tensor(bits).float()

def moves_2_str(moves):
    s = ""
    for m in moves:
        s += str(m) + " "
    return s + "\n"

In [15]:
import torch
import torch.nn as nn


class NN(torch.nn.Module):

    def __init__(self):
        super().__init__()

        self.l1 = nn.Linear(832, 512)
        self.l2 = nn.Linear(512, 256)
        self.l3 = nn.Linear(256, 128)
        self.l4 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        x = self.l4(x)
        return x

In [16]:
# count and weight the material for each side, a very simple evaluation metric
def static_material_eval(board: chess.Board) -> float:
    # 1 - pawn, 3 - knight, bishop, 5 - rook, 9 - queen, 50 - king
    type_value = [1, 3, 3, 5, 9, 50]
    
    score = 0
    for i in range(6):
        score += bin(int(board.pieces(i + 1, chess.WHITE))).count("1") * type_value[i] # count the number of 1s and weight with the piece value
        score -= bin(int(board.pieces(i + 1, chess.BLACK))).count("1") * type_value[i]
    return score

# return the score of the position - black, + white and if the game is over
def eval_position(board: chess.Board, nn) -> (float, bool):
    
    if board.is_game_over():
        if board.is_checkmate():
            # if white is checkmate black gets an -infinit score and inf for black in checkmate
            return (float("-inf"), True) if board.turn else (float("inf"), True)
        # 0 score for a draw
        return 0, True

    # if the game is not over evaluate the board using the nn
    return nn.forward(board_2_tensor(board)), False
    # return static_material_eval(board), False

In [17]:
# a minmax tree search of each possible move util the max depth is reached using alpha beta pruning
def minmax(board: chess.Board, model: NN, depth: int, alpha: float, beta: float, is_maximizing: bool) -> (float, chess.Move):
    
    score, is_gameover = eval_position(board, model)
    if is_gameover or depth is 0:
        # stop the tree search and return, no move is returned because there are no legals or we stop exploring here
        return score, []

    best_moves = []
    best_score = 0
    # white wants to maximize the score
    if is_maximizing:
        best_score = float("-inf")

        for move in board.generate_legal_moves():
            board.push(move)
            score, next_moves = minmax(board, model, depth - 1, alpha, beta, False)
            board.pop()

            if score >= best_score:
                best_score = score
                best_moves = [move] + next_moves
            alpha = max(alpha, score)

            if alpha >= beta:
                break

    # black wants to minimize the score
    else:
        best_score = float("inf")
        
        for move in board.generate_legal_moves():
            board.push(move)
            score, next_moves = minmax(board, model, depth - 1, alpha, beta, True)
            board.pop()

            if score <= best_score:
                best_score = score
                best_moves = [move] + next_moves
            beta = min(beta, score)

            if beta <= alpha:
                break

    return best_score, best_moves

def find_best_move(board, model, depth):
    move = None
    for i in range(depth + 1):
        score, move = minmax(board, model, i, float("-inf"), float("inf"), board.turn)
        if score in [float("-inf"), float("inf")]:
            return score, move
    return score, move

In [38]:
def play_game(depth: int, model: NN):
    b = chess.Board()
    positions = np.empty([0, 832])

    game_over = False
    i = 0
    while not game_over:
        positions = np.append(positions, board_2_tensor(b))

        score, moves = find_best_move(b, model, 2)
        b.push(moves[0])
        game_over = b.is_game_over()
        i += 1
    result, _ = eval_position(b, model)
    return positions, result

In [36]:
64 * 13 * b.ply()

840320